This notebook mimics the myc_notebook from pyetm v1 to demonstrate how functionalities remain the same, despite being dressed up differently.

**Multi-year charts**

The MYC function within the ETM allows users to view and edit multiple scenarios at once. This 
feature is useful to compare different scenarios or to built a transition path that contains 
scenarios with different end years, see https://myc.energytransitionmodel.com/


**Model Initialisation**

Where you used to initialize the 'model' with the `model.from_excel()` function, now we work with a 'scenario' or 'scenarios' (TBC - what will we call the multiple scenario object?).

You need to run the setup_notebook() command to get the API ready - this will also validate that you've set your API token correctly. You determine whether you connect with the beta, stable or live version of the ETM via the ETM_URL in the config.

In [ ]:
from example_helpers import setup_notebook
from pyetm.models import Scenario

setup_notebook()

Then, you can create a scenario (or scenarios) with the Scenario.from_excel. For now this functionality is not implemented, so the example will continue by loading an individual scenario. Note that when multi-scenario functionality has been implemented, these same steps should work exactly the same way, but on the Scenarios object, rather than the Scenario object.

In [ ]:
scenario = Scenario.load(2690499)

In [ ]:
# Get scenario input parameters (incl. units)
inputs = scenario.inputs
print(inputs.to_dataframe().head())

You will notice that in the original pyetm this call was:
```
inputs = model.get_parameters(exclude=True)
inputs.head()
```
Pyetm v2 is more developer friendly, which means that each of the 'objects' on the model (for example the inputs) are still python objects until you call **to_dataframe()** on them. This is one of the more significant changes - you will have to specify (in the notebooks) what format you want the data in. For the flows from Excel --> ETM and ETM --> Excel, this is all taken care of internally.

In [ ]:
# We also distinguish between inputs and user_values. Usually user values are more interesting:
user_values = scenario.user_values()
user_values

In [ ]:
# Execute gqueries to fetch data from ETM (incl. units)
gqueries = scenario.results()
print(gqueries)

There are no default gqueries, you need to specify them yourself - hence the empty dataframe .

In [ ]:
scenario.add_queries(["dashboard_emissions"])
gqueries = scenario.results()
print(gqueries)

In [ ]:
# Optional: Fetch hourly electricity price profiles from ETM
custom_curves = scenario.custom_curves_series()
for key in custom_curves: print(key)

#TODO: Discuss

Do you want a convenient way to only get the custom curves related to electricity price? Or are you more interested in the carrier curves? 

In [ ]:
# Optional: Fetch hourly carrier profiles from ETM
output_curves = scenario.get_output_curves(carrier_type='electricity')
output_curves

As you can see, this returns a dictionary of dataframes, including the merit_order, electricity_price and residual_load dataframes. If you want a specific output, just request it:

In [ ]:
electricity_price = scenario.output_curve(curve_name='electricity_price')
electricity_price

**Export scenarios to Excel**

Just like with the original implementation, everything in the scenario can be exported to an excel file. This is done via a "Scenario Packer" which organises the data usefully to display via excel.

This example is simplified in the sense that we just add everything in the scenario to the packer, but you can also add specific inputs, gqueries curves etc to a packer (see more examples in the scenario_to_excel notebook).

In [ ]:
from pyetm.models import ScenarioPacker

packer = ScenarioPacker()
packer.add(scenario)
packer.to_excel('my_excel.xlsx')